## Anomaly Detection Basic

##### 라이브러리 로드

In [1]:
# 데이터 처리 
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 딥러닝 / 머신러닝 프레임워크
from sklearn.model_selection import train_test_split # 훈련/테스트 데이터 나누기
from sklearn.preprocessing import MinMaxScaler # 스케일링
from sklearn.impute import KNNImputer # 결측치 처리 - KNN 기법 사용

import tensorflow as tf # 

In [3]:
# 사용자 정의 시각화 및 레포트 함수
from lib.vis import autoencoder_report, visualize_reconstruction
# 모델 정의
from lib.ae import build_basic_ae

### 1) 오토인코더 기본 모델링 ( ECG 데이터 이상 탐지 )

##### 데이터셋(ECG데이터) 로드

In [4]:
# 데이터셋 다운로드 경로 확인 및 생성
dataset_dir = 'dataset'
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir, exist_ok=True)


In [ ]:
# ECG 데이터 다운로드 (파일이 없는 경우에만)
ecg_file = os.path.join(dataset_dir, 'ecg.csv')
if not os.path.exists(ecg_file):
    print("ECG 데이터 다운로드 중...")
    url = "https://storage.googleapis.com/download.tensorflow.org/data/ecg.csv"
    
    # 데이터 다운로드
    data = pd.read_csv(url)
    data.to_csv(ecg_file, index=False)
    print("다운로드 완료!")
else:
    print("ECG 데이터가 이미 존재합니다.")


In [ ]:
data = pd.read_csv('dataset/ecg.csv')
print(data.head())
print(data.shape)

##### 데이터 전처리

In [ ]:
# 데이터 확인
data.head()

In [8]:
# 레이블 0과 1 변환
data.iloc[:, -1] = data.iloc[:, -1].apply(lambda x: 1 if x == 1 else 0)

In [ ]:
 # na 값 확인
data.isnull().sum()

In [ ]:
# KNN 방법 사용
print("\nKNN을 사용한 결측치 처리:")
knn_imputer = KNNImputer(n_neighbors=5)
data_knn = pd.DataFrame(knn_imputer.fit_transform(data), columns=data.columns)

# 결측치 처리 결과 확인
print(f"KNN 결측치 수: {data_knn.isnull().sum().sum()}")

# KNN 방법을 선택하여 데이터 업데이트
data = data_knn

# 결측치 처리 전후 비교
print("\n처리 전후 데이터 형태 비교:")
print(f"원본 데이터 형태: {data.shape}")
print(f"처리 후 데이터 형태: {data_knn.shape}")


In [11]:
# 데이터 분리
x = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [ ]:
# 이상 데이터 비율 확인
print(y.value_counts())
print(y.value_counts() / y.shape[0])

In [ ]:
# 훈련, 테스트 데이터 나누기
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .2, random_state=2025)
x_train.shape, y_train.shape

In [14]:
# 스케일링
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

In [ ]:
# 학습을 위해서는 Normal 데이터만 이용한다.
x_train_normal = x_train[y_train == 1]
x_train_normal.shape

##### 모델링

In [ ]:
# 파라미터 설정 (여러분들이 조정해보세요)
epochs = 100
batch_size = 64
input_dim = x_train_normal.shape[1] # 입력 레이어 차원은 필드의 수와 같다.
input_dim

In [ ]:
autoencoder = build_basic_ae(input_dim)

In [ ]:
# 모델 훈련
# 조기종료 콜백 추가 : 너무 좋은 성능이 나오면 더 훈련하지 않고 종료)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

history = autoencoder.fit(
    x=x_train_normal, 
    y=x_train_normal,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(x_val, x_val),
    callbacks=[early_stopping],
    verbose=1
).history

##### 

##### 모델 평가

In [ ]:
# 이상 데이터 예측
pred = autoencoder.predict(x_val)
pred

In [ ]:
# 100번째 인덱스 데이터의 입력 및 재구성 데이터 대조 시각화
visualize_reconstruction(x_val, pred, 100)


In [ ]:
# 레포트 생성
autoencoder_report(x_val, pred, y_val, 0.01, history)